# Example: MP573 (Mathematical Methods in Medical Physics)

### Table of Contents
1. [Section 1: Fourier Analysis of 1D Signals](#section-1-fourier-analysis-of-1d-signals)
2. [Section 2: Basic Optimization (Independent Practice)](#section-2-basic-optimization-independent-practice)

---

## Section 1: Fourier Analysis of 1D Signals
The Fourier transform is a fundamental mathematical tool that plays a crucial role in signal analysis! You will use it quite often in MP573. It allows us to decompose a signal into the frequencies that make it up (or vice-versa).

Some specific uses include:
- Signal processing: It allows us to filter out noise, enhance specific frequencies, and reconstruct cleaner images or signals.
- Image reconstruction: In techniques like MRI and CT, the Fourier transform is essential for converting raw data into meaningful images.
- Data compression: By focusing on the most significant frequency components, we can compress medical data for more efficient storage and transmission.
- System characterization: The Fourier transform helps us understand the frequency response of systems.
- and many, many more!

This toy example will cover basic noise filtering of a signal by thresholding its frequency spectrum.

#### Import useful packages

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

#### Generate and plot an example signal with two frequencies

In [ ]:
t = np.linspace(-50, 50, 1001)
freq1, freq2 = 0.1, 0.2

signal = np.sin(2*np.pi*freq1*t) + np.sin(2*np.pi*freq2*t) 

plt.figure(figsize=(10, 5))
plt.plot(t, signal)
plt.title('Signal with two frequencies')
plt.show()

#### Signals are usually noisy, so let's add some noise

In [ ]:
noise_mean, noise_std = 0, 1
noise = np.random.normal(noise_mean, noise_std, signal.shape)

noisy_signal = signal + noise

plt.figure(figsize=(10, 5))
plt.plot(t, noisy_signal)
plt.title('Noisy signal')
plt.show()

#### Signal analysis in the frequency domain

In [ ]:
# The Fourier transform of the signal will show the frequency spectrum of the signal
# We can estimate the original signal by filtering the noisy signal in the frequency domain
signal_fft = np.fft.fftshift(np.fft.fft(noisy_signal))
freqs = np.fft.fftshift(np.fft.fftfreq(t.shape[-1], t[1] - t[0]))

plt.figure(figsize=(10, 5))
plt.plot(freqs, np.abs(signal_fft))
plt.xlim(-1, 1)
plt.xlabel('Frequency')
plt.ylabel('Amplitude')
plt.title('Frequency spectrum of the noisy signal')
plt.show()


#### Filter out low-power signal components using simple thresholding

In [ ]:
# We can use indexing and boolean expressions to find the indices of the frequencies we want to keep
peak_indices = np.abs(signal_fft) > 100
filtered_fft = signal_fft.copy()
filtered_fft[~peak_indices] = 0

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(freqs, np.abs(filtered_fft))
plt.axhline(100, color='red', linestyle='--')
plt.scatter(freqs[peak_indices], np.abs(filtered_fft)[peak_indices], color='red', marker='x')
plt.xlim(-1, 1)
plt.xlabel('Frequency')
plt.ylabel('Amplitude')
plt.title('Frequency spectrum of the filtered signal')
plt.show()

#### Convert the filtered signal back to the time domain

In [ ]:
# The inverse Fourier transform will give us the filtered signal
filtered_signal = np.fft.ifft(np.fft.ifftshift(filtered_fft))

plt.figure(figsize=(10, 5))
plt.plot(t, np.real(filtered_signal))
plt.title('Filtered signal')
plt.show()

---

## Section 2: Basic Optimization (Independent Practice)

Optimization is another key medical physics concept that you will cover in MP573! It involves finding the best possible solution to a problem within given constraints. 

In medical physics, optimization is applied in numerous areas, including:
- Image reconstruction: Improving image quality by finding the best parameters for reconstruction algorithms in CT, MRI, and other imaging modalities.
- Treatment planning: Determining the optimal arrangement of radiation beams or radioactive sources to achieve the desired therapeutic effect.
- Diagnostic imaging: Balancing image quality with radiation dose in X-ray and CT imaging.
- and much, much more!

Typically, our search spaces are large and complex, requiring iterative optimization methods. These methods often involve defining an objective function (sometimes called a cost function or fitness function) that quantifies the quality of a solution, and then systematically searching for the values that minimize or maximize this function. This toy example illustrates how we might use gradient descent, a common optimization algorithm (and the cornerstone of most AI training algorithms!), to minimize a simple function.

## Problem
##### Find the parameters (x, y) that minimize the following objective function
$$f(x, y) = (x-2)^2 + (y-3)^2$$


##### For our gradient descent algorithm, we need to know the gradient of our objective function.
$$\nabla f(x, y) = \begin{bmatrix} \frac{\partial f}{\partial x} \\ \frac{\partial f}{\partial y} \end{bmatrix} = \begin{bmatrix} 2(x-2) \\ 2(y-3) \end{bmatrix}$$

#### Import useful packages

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm # tqdm is a library that provides a progress bar around for loops - useful for monitoring long-running processes

#### Define the objective function (cost function) and its gradient

In [ ]:
# Objective function to minimize
def objective_function(x, y):
    return (x - 2)**2 + (y - 3)**2

# Gradient of the objective function
def gradient(x, y):
    dx = 2 * (x - 2)
    dy = 2 * (y - 3)
    return np.array([dx, dy])

#### Define our basic gradient descent optimization function

In [ ]:
# Gradient descent evaluates the gradient at the current point and takes a step in the direction opposite to the gradient
# The step size is determined by the learning rate
# The process is repeated for a fixed number of iterations
# After many iterations, the point will converge to the minimum of the objective function
def gradient_descent(start_point, learning_rate, num_iterations):
    path = [start_point]
    point = np.array(start_point)
    
    for _ in tqdm(range(num_iterations)):
        grad = gradient(point[0], point[1])
        point = point - learning_rate * grad
        path.append(point)
    
    return np.array(path)

#### Now, set up and run our optimization function

In [ ]:
# Set up the optimization
start_point = [-1, 6]
learning_rate = 0.1
num_iterations = 50

# Run the optimization
optimization_path = gradient_descent(start_point, learning_rate, num_iterations)

#### Visualize the optimization path and the result!

In [ ]:
# Create a grid of points
x = np.linspace(-2, 6, 100)
y = np.linspace(-2, 8, 100)
X, Y = np.meshgrid(x, y)
Z = objective_function(X, Y)

# Plot the results
plt.figure(figsize=(12, 8))
plt.contour(X, Y, Z, levels=250, cmap='viridis')
plt.colorbar(label='Objective Function Value')
plt.plot(optimization_path[:, 0], optimization_path[:, 1], 'ro-', linewidth=1.5, markersize=5)
plt.plot(2, 3, 'g*', markersize=15, label='Global Minimum')
plt.title('Gradient Descent Optimization')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.grid(True)
plt.show()

#### Print the final optimized values

In [ ]:
print(f"Optimized solution: x = {optimization_path[-1, 0]:.4f}, y = {optimization_path[-1, 1]:.4f}")
print(f"Minimum objective function value: {objective_function(*optimization_path[-1]):.4f}")